In [ ]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# Генерация синтетических данных (замените на свои данные)
X, y = make_classification(n_samples=10000, n_features=20, n_classes=2, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение функции для оценки модели
def objective(trial):
    # Подбор гиперпараметров с помощью Optuna
    param = {
        'objective': 'binary:logistic',  # Задача бинарной классификации
        'eval_metric': 'auc',            # Используем AUC для оценки
        'tree_method': 'auto',           # Метод построения деревьев
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),  # Тип бустинга
        'max_depth': trial.suggest_int('max_depth', 3, 10),  # Глубина дерева
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 0.1),  # Скорость обучения
        'n_estimators': trial.suggest_int('n_estimators', 50, 2000),  # Число деревьев
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),  # Подвыборка
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),  # Количество признаков для каждого дерева
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),  # Минимальный вес для разделения
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),  # Важность положительного класса
    }
    
    # Обучение модели с текущими гиперпараметрами
    model = xgb.XGBClassifier(**param)
    model.fit(X_train, y_train)
    
    # Прогнозирование на тестовой выборке
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Рассчитываем метрику AUC
    auc = roc_auc_score(y_test, y_pred_proba)
    
    # Можно также вычислить Gini-коэффициент как 2*AUC - 1
    gini = 2 * auc - 1
    
    return gini  # или auc, если вы предпочитаете работать с AUC

# Запуск подбора гиперпараметров с помощью Optuna
study = optuna.create_study(direction='maximize')  # Оптимизируем по Gini (или AUC)
study.optimize(objective, n_trials=50)  # Число проб - можно увеличить для более точного подбора

# Выводим лучшие гиперпараметры и результат
print("Best trial:", study.best_trial.params)
print("Best Gini:", study.best_value)

# Обучение модели с лучшими гиперпараметрами
best_model = xgb.XGBClassifier(**study.best_trial.params)
best_model.fit(X_train, y_train)

# Оценка модели
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
gini = 2 * auc - 1
print(f"Test AUC: {auc:.4f}")
print(f"Test Gini: {gini:.4f}")

# Опционально: Печать матрицы ошибок
y_pred = best_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
